In [20]:
import os
import json

vul = "RE"
dir_vul = f"../../data/processed_data/{vul}_SolidiFi"
dir_contract = os.path.join(dir_vul, "contracts")
dir_LOCs = os.path.join(dir_vul, "LOCs")

for i in range(1, 51):
    LOCs_path = os.path.join(dir_LOCs, f"{i}.json")
    contract_path = os.path.join(dir_contract, f"{i}.sol")

    # Read all lines from the Solidity contract
    with open(contract_path, "r") as f:
        contract_lines = f.readlines()

    # Read JSON list of dictionaries
    with open(LOCs_path, "r") as f:
        data = json.load(f)

    # Process each dictionary entry
    for entry in data:
        line_range = entry.get("vulnerableLines", "")
        vulnerable_code = []

        if line_range:
            # Support multiple ranges separated by commas (e.g., "6-8,10-12")
            ranges = line_range.split(",")
            for r in ranges:
                if "-" in r:
                    start, end = map(int, r.strip().split("-"))
                else:
                    start = end = int(r.strip())

                # Keep original spacing, just convert tabs to 4 spaces
                vulnerable_code.extend([
                    line.rstrip("\n").replace('\t', '    ') for line in contract_lines[start - 1:end]
                ])

        # Add the processed vulnerable code to the dictionary
        entry["vulnerableCode"] = vulnerable_code

    # Save updated JSON
    with open(LOCs_path, "w") as f:
        json.dump(data, f, indent=4)
